# develop and test clean_db.py

In [1]:
import src.clean_db as cdb
import src.metadata_handler as mdh

%autoreload 2

In [68]:
import pandas as pd
import numpy as np

import re

In [14]:
raw_db = pd.read_json('data/test_DB.json')

In [91]:
raw_db.head(3).T

,0,1,2
by-line,Cathy Clark,Picasa,Cathy Clark
caption/abstract,M2E32L107-108R390B311,NaN,1
contact,[],[],[]
copyright notice,Conservation Northwest,NaN,Conservation Northwest
date created,2.01505e+07,NaN,2.01506e+07
file_path,first_sample/EK000004-2.JPG,first_sample/EK000004.JPG,first_sample/EK000010-2.JPG
keywords,[Camera Check],[camera check],[hoary marmot]
object name,wolverine,NaN,wolverine
sub-location,Blackjack Ridge-2015-1,NaN,Blackjack Ridge-2015-1
supplemental category,[],[],[]


# Rename Columns (remove spaces)

In [30]:
raw_db.columns

Index([u'by-line', u'caption/abstract', u'contact', u'copyright notice',
       u'date created', u'file_path', u'keywords', u'object name',
       u'sub-location', u'supplemental category', u'time created'],
      dtype='object')

In [38]:
def space_free_col_names(db):
    dd = {}
    for col in db.columns:
        new_col = col
        new_col = new_col.replace(' ', '_')
        new_col = new_col.replace('-', '_')
        new_col = new_col.replace('/', '_')
        if new_col != col:
            dd[col] = new_col
    db = db.rename(columns = dd)
    return db

In [40]:
db = raw_db

print 'original cols: \n',db.columns

db = space_free_col_names(db)

print 'new cols: \n',db.columns

original cols: 
Index([u'by-line', u'caption/abstract', u'contact', u'copyright notice',
       u'date created', u'file_path', u'keywords', u'object name',
       u'sub-location', u'supplemental category', u'time created'],
      dtype='object')
new cols: 
Index([u'by_line', u'caption_abstract', u'contact', u'copyright_notice',
       u'date_created', u'file_path', u'keywords', u'object_name',
       u'sub_location', u'supplemental_category', u'time_created'],
      dtype='object')


# Handle Dates and Times

In [ ]:
# change type of entries in date_created to int?

In [88]:
def _fix_date(date):
    # remove decimal?
    date = str(date)
    if date == 'nan':
        return 'NaN'
    else:
        return re.sub('(....)(..)(..)(..)', '\\1-\\2-\\3', date)

fix_date = np.vectorize(_fix_date)

In [93]:
db.date_created = fix_date(db.date_created)
db.head(3).T

,0,1,2
by_line,Cathy Clark,Picasa,Cathy Clark
caption_abstract,M2E32L107-108R390B311,NaN,1
contact,[],[],[]
copyright_notice,Conservation Northwest,NaN,Conservation Northwest
date_created,2015-05-30,NaN,2015-06-04
file_path,first_sample/EK000004-2.JPG,first_sample/EK000004.JPG,first_sample/EK000010-2.JPG
keywords,[Camera Check],[camera check],[hoary marmot]
object_name,wolverine,NaN,wolverine
sub_location,Blackjack Ridge-2015-1,NaN,Blackjack Ridge-2015-1
supplemental_category,[],[],[]


In [102]:
def _fix_time(time):
    # remove decimal?
    time = str(time)
    if time == 'nan':
        return None
    elif len(time) == 7:
        return re.sub('(.)(..)(..)(..)', '\\1:\\2:\\3', time)
    else:
        return re.sub('(..)(..)(..)(..)', '\\1:\\2:\\3', time)

fix_time = np.vectorize(_fix_time)

In [103]:
fix_time(db.time_created)

array(['11:49:52', 'None', '14:54:40', 'None', '2:54:40', 'None',
       '2:54:41', 'None', '2:54:46', 'None', '19:56:03', 'None', '7:04:24',
       'None', '19:04:25', 'None', '19:04:26', 'None', '19:04:29', 'None',
       '19:04:30', 'None', '19:04:30', 'None', '19:04:25', 'None',
       '19:04:26', 'None', '19:04:29', 'None', '19:04:30', 'None',
       '19:04:30', 'None'], 
      dtype='|S8')

# Handle null values
## (and Clean Obvious Duplicates?)

THOUGHTS:

drop anything where 
+ ? improper/nonexistent keywords
+ ? by_line: Picasa (or similar)
+ ? by_line NOT person name
+ ? object / copyright / sub_loc == NaN


In [106]:
db[['caption_abstract', 'file_path', 'keywords']]

,caption_abstract,file_path,keywords
0,M2E32L107-108R390B311,first_sample/EK000004-2.JPG,[Camera Check]
1,NaN,first_sample/EK000004.JPG,[camera check]
2,1,first_sample/EK000010-2.JPG,[hoary marmot]
3,NaN,first_sample/EK000010.JPG,[marmot]
4,1,first_sample/EK000011-2.JPG,[hoary marmot]
5,NaN,first_sample/EK000011.JPG,[marmot]
6,1,first_sample/EK000012-2.JPG,[hoary marmot]
7,NaN,first_sample/EK000012.JPG,[marmot]
8,1,first_sample/EK000013-2.JPG,[hoary marmot]
9,NaN,first_sample/EK000013.JPG,[marmot]
